In [47]:
import subprocess
import sys
import os
import shutil
import rdkit
from rdkit import Chem
# this is a conformation generation script based on RDKit and Merck Molecular Mechanics that takes SMILES as input
import conf_gen

In [48]:


def protonate_smiles(neutral_smiles):
    protonated_smiles=[]
    for smile in neutral_smiles:
        # create molecule from SMILES
        mol = Chem.MolFromSmiles(smile)
        # find index of nitrogen
        nIdx = mol.GetSubstructMatch(Chem.MolFromSmarts('N'))
        # set the nitrogen atom's formal charge to 1
        mol.GetAtomWithIdx(*nIdx).SetFormalCharge(1)
        # print the protonated SMILES
        protonated_smiles.append(Chem.MolToSmiles(mol))
        
    return protonated_smiles  




In [49]:

"""this function takes a file named by smile string (smile+"_confs.sdf") and 
returns numbers of conformers written to that file"""

def find_num_conformers_in_a_file(smile,keyword):
    k=0
    with open(smile+"_confs.sdf", 'r') as f:
        for line in f:
            words = line.split()
            if keyword in words:
                k+=1
    return k




"""this function takes a file name smile_confs.sdf and splits them and returns a dictionary of conformer id 
and energy"""

def conformers_energy(smile):
    N_conf=find_num_conformers_in_a_file(smile,"RDKit")
    lines = open(smile+"_confs.sdf", 'r').readlines()
    num_of_lines = int(len(lines)/N_conf)
    my_list = range(0, len(lines), num_of_lines)
    
    conf_energy={}
    for i, j in enumerate(my_list):
        f2 = open(smile+'_c'+str(i+1)+'.sdf', 'w')
        for L in range(j, j+num_of_lines):
            f2.write(str(lines[L]))
            if "conformer_id" in lines[L]:
                conf_energy[lines[L+1].split()[0]]=lines[L+4].split()[0]
        
    f2.close()
    return conf_energy,min(conf_energy, key=conf_energy.get)

In [50]:
def write_gaussian_input(calc_type, inputsmile,charge):
    with open(inputsmile + '.com', 'w') as f:
        
        # one can change multipilicty and CPU, Memory requirement here    
        f.write('%NProcShared=4\n')
        f.write('%Mem=30GB\n')

        # write Gaussian input header for IG frequency calculations
        if calc_type=='IG_freq':
            f.write('#n G4 Opt=tight Freq pressure=0.987 temp=298.15\n')
        elif calc_type=='HFE_continum':
            f.write('#n M062X/6-31+G(d,p) Opt=(MaxCyc=250) freq SCF=(MaxCyc=250, Tight) Integral=UltraFine SCRF=(SMD, solvent=water) pressure=0.987 temp=298.15\n')
  
        elif calc_type=='HFE_vaccum':
            f.write('#n M062X/6-31+G(d,p) Opt=tight Freq pressure=0.987 temp=298.15\n')
        else:
            print('select valid calculations')

        f.write('\n')
        f.write(inputsmile+'\n')
        f.write('\n')
        f.write(str(charge)+"  "+  '1\n')
        # append obabel generated gaussian coordinate section 
        ID= conformers_energy(inputsmile)[1]  # this is the id of minimum energy conformer
        std_out, std_err=subprocess.Popen(['babel', '-isdf', inputsmile+ "_c" + ID + '.sdf', '-ocom'], stdout=subprocess.PIPE).communicate()
        coords=std_out.decode("utf-8")
        f.writelines( "%s\n" % line for line in coords.splitlines()[5:] )


In [64]:
neutral_smile_strings=['C(CO)N','C(CO)NCCO','C(CO)N(CCO)CCO']
protonate_smile_strings=protonate_smiles(protonate_smile_strings)
conf_gen.main(neutral_smile_strings)
conf_gen.main(protonate_smile_strings)

Molecule 1 : generated 99 conformers and 1 clusters
Molecule 2 : generated 100 conformers and 1 clusters
Molecule 3 : generated 100 conformers and 2 clusters
Molecule 1 : generated 99 conformers and 1 clusters
Molecule 2 : generated 100 conformers and 1 clusters
Molecule 3 : generated 100 conformers and 2 clusters


In [65]:
for i,j in dict(zip(neutral_smile_strings,protonate_smile_strings)).items():
    write_gaussian_input('IG_freq', i, 0)
    write_gaussian_input('IG_freq', j, 1)

os.mkdir('IG_freq')
sourcepath=os.getcwd()
sourcefiles = os.listdir(sourcepath)
destinationpath = os.path.join(sourcepath, 'IG_freq')
for file in sourcefiles:
    if file.endswith('.com'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))





In [66]:
for i,j in dict(zip(neutral_smile_strings,protonate_smile_strings)).items():
    write_gaussian_input('HFE_continum', i, 0)
    write_gaussian_input('HFE_continum', j, 1)

os.mkdir('HFE_continum')
sourcepath=os.getcwd()
sourcefiles = os.listdir(sourcepath)
destinationpath = os.path.join(sourcepath, 'HFE_continum')
for file in sourcefiles:
    if file.endswith('.com'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))



In [67]:
for i,j in dict(zip(neutral_smile_strings,protonate_smile_strings)).items():
    write_gaussian_input('HFE_vaccum', i, 0)
    write_gaussian_input('HFE_vaccum', j, 1)

os.mkdir('HFE_vaccum')
sourcepath=os.getcwd()
sourcefiles = os.listdir(sourcepath)
destinationpath = os.path.join(sourcepath, 'HFE_vaccum')
for file in sourcefiles:
    if file.endswith('.com'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))
        



In [68]:
# remove all redundant sdf files in current working directory and only keep Gaussian input files
dir_name = os.getcwd()
test = os.listdir(dir_name)

for item in test:
    if item.endswith(".sdf"):
        os.remove(os.path.join(dir_name, item))
    


    
    
